### declare system parameters

In [1]:
import numpy as np
np.set_printoptions(precision=5, suppress=True)

In [2]:
a = 1
b = 1
dt = 0.5

### system dynamics

In [3]:
x_prior = np.array([np.deg2rad(30), np.deg2rad(60)])

sigma_alpha = np.deg2rad(5)
sigma_beta = np.deg2rad(10)
rho = -0.7
S_x_prior = np.array([[sigma_alpha**2, rho*sigma_alpha*sigma_beta],
                       [rho*sigma_alpha*sigma_beta, sigma_beta**2]])

In [4]:
Gx = np.eye(2)
Gu = np.array([[dt, 0],
               [dt, dt]])

### current state and variances

In [5]:
omega1 = 10 * 2* np.pi / 60  # rad/s
omega2 = 5 * 2* np.pi / 60  # rad/s
u = np.array([omega1, omega2])
sigma_u = 1 * 2 * np.pi / 60  # rad/s
S_u = np.array([[sigma_u**2, 0],
                [0, sigma_u**2]])

z_now = np.array([np.deg2rad(60), np.deg2rad(70), 0.8, 1.8])
sigma_z12 = np.deg2rad(5)
rho_z12 = 0.4
sigma_z34 = 0.10
rho_z34 = 0.3
S_z = np.array([[sigma_z12**2, rho_z12*sigma_z12**2, 0, 0],
                    [rho_z12*sigma_z12**2, sigma_z12**2, 0, 0],
                    [0, 0, sigma_z34**2, rho_z34*sigma_z34**2],
                    [0, 0, rho_z34*sigma_z34**2, sigma_z34**2]])

### predictiction step

In [6]:
x_bar = Gx @ x_prior + Gu @ u
S_xbar = Gx @ S_x_prior @ Gx.T + Gu @ S_u @ Gu.T

### Kalman Gain

In [7]:
# calculate Hx on xbar
alpha, beta = x_bar
Hx = np.array([[1, 0],
               [0, 1],
               [-a*np.sin(alpha), -b*np.sin(beta)],
               [a*np.cos(alpha), b*np.cos(beta)]])

K = S_xbar @ Hx.T @ np.linalg.inv(Hx @ S_xbar @ Hx.T + S_z)
K

array([[ 0.41563, -0.22306, -0.2353 ,  0.20607],
       [-0.29315,  0.59084, -0.30374, -0.02386]])

### Innovation step

In [8]:
z_bar = np.array([x_bar[0],
                   x_bar[1], 
                   a*np.cos(x_bar[0]) + b*np.cos(x_bar[1]),
                   a*np.sin(x_bar[0]) + b*np.sin(x_bar[1])])
x_next = x_bar + K@(z_now - z_bar)
np.rad2deg(x_next)

array([59.89631, 74.63918])

In [9]:
S_xnext = (np.eye(2) - K@Hx)@S_xbar
S_xnext

array([[ 0.00249, -0.00043],
       [-0.00043,  0.00361]])